In [1]:
import pandas as pd
import numpy as np
from telethon.sync import TelegramClient
from telethon.tl.types import PeerUser, PeerChat, PeerChannel, InputMessagesFilterPhotoVideo
from telethon.tl.functions.channels import GetFullChannelRequest
import json
import re
from telethon.errors import ChannelPrivateError
import pickle

/var/folders/r3/t2r07q612snc9xmhh7_6wkqm0000gn/T/ipykernel_25725/3674494022.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
id_centro_studi = 1344471586
id_bloccco = 1394864973

In [3]:
with open('../credentials/credentials.pickle','rb') as file:
    api_id, api_hash, phone, username = pickle.load(file)

Let's start building a dictionnaire id_to_names and id_to_description.
We start from 'forward_to' of the first edgelist (there will be only Azione Incel and FDB)

In [4]:
id_to_names = {}
id_to_description = {}

In [5]:
import datetime

In [6]:
offset_date = datetime.datetime(2023,10,1)

In [69]:
id_to_names, id_to_description

({1344471586: 'Centro Stvdi Fascismo ▐┛🏴', 1394864973: 'Blocco Studentesco'},
 {1344471586: 'Telegram https://t.me/CentroStvdiFascismo\nSito https://centrostvdifascismo.wordpress.com\nCinematografo https://odysee.com/@CentroStvdiFascismo\nWhatsapp https://whatsapp.com/channel/0029VaIejYSF1YlXO33jX32X\nContatti centrostudifascismo@virgilio.it',
  1394864973: "Canale ufficiale del movimento studentesco più rivoluzionario d'Italia - dal 2006"})

In [8]:
async def get_data_channel(channel_id, offset_date, names_dict, desc_dict):
    """
    Get all messages from channel NEWER than offset date (e.g. sent after offset date)

    args:

    channel_id: id of channel to be scraped (has to be an integer!)
    offset_date: date limit 
    names_dict: dict of id-to-names
    desc_dict: dict of id-to-bio
    
    return list of patcthed messages
    """
    
    async with TelegramClient(username, api_id, api_hash) as client:
                entity = await client.get_entity(PeerChannel(channel_id))
                full_entity = await client(GetFullChannelRequest(channel=entity))

                names_dict[channel_id] = entity.title #get group name
                desc_dict[channel_id] = full_entity.full_chat.about #get group bio

                lista_mex = await client.get_messages(entity, reverse=True, offset_date=offset_date, limit=None)
    
    return lista_mex

In [9]:
def get_channels_forward(lista_messaggi, receving_id):
    """
    lista_messaggi: list of patched telethon messages
    receving_id: id of the scraped group. So the forwarded_to id

    returns list of tuple with format (forwarded_from, forwarded_to)
    where forwarded_to is the receving_id

    """
    lista_fwd = []
    for mex in lista_messaggi:
        dict_mex = mex.to_dict()
        if 'fwd_from' in dict_mex.keys() and dict_mex['fwd_from'] is not None:
            fwd_info = dict_mex['fwd_from']['from_id']
            if type(fwd_info) == dict and fwd_info['_']=='PeerChannel':
                lista_fwd.append((fwd_info['channel_id'], receving_id))

    return lista_fwd
        

# Get data from Blocco studentesco and centro stvdi fascismo

In [49]:
first_edgelist_list = []
for codice in (id_centro_studi, id_bloccco):
    try:
        list_messages = await get_data_channel(int(codice), offset_date, id_to_names, id_to_description)
        list_tuples = get_channels_forward(list_messages, int(codice))
        first_edgelist_list.extend(list_tuples)
    except(ChannelPrivateError, ValueError) as e:
        print(e)

    

In [71]:
forward_from = list(map(lambda x: x[0], first_edgelist_list))
forward_to = list(map(lambda x: x[1], first_edgelist_list))

In [72]:
first_edgelist_df_raw = pd.DataFrame({'forward_from':forward_from,\
    'forward_to':forward_to})

In [75]:
#save edgelist
first_edgelist = first_edgelist_df_raw.value_counts().reset_index()
first_edgelist.to_csv('../first_edgelist/first_edgelist.csv',index=False)

In [78]:
#need to cast keys as int (instead of int64) to dump json
id_to_names_int = dict(map(lambda x: (int(x[0]),x[1]),id_to_names.items()))
id_to_desc_int = dict(map(lambda x: (int(x[0]),x[1]),id_to_description.items()))

In [79]:
with open('../dictionaires/id_to_names.json','w') as file_names, \
    open('../dictionaires/id_to_desc.json','w') as file_desc:
    json.dump(id_to_names_int, file_names)
    json.dump(id_to_desc_int, file_desc)